In [ ]:
import sqlite3
from typing import Union, Any
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
graph_track_features = [
    'danceability',
    'energy',
    'loudness',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'duration_ms']

categorical_track_features = [
    'key',
    'mode',
    'time_signature'
]

In [ ]:
con = sqlite3.connect('spotify.db')
track_features = pd.read_sql("""SELECT * from songs""", con)
con.close()

In [ ]:
track_features.head()

In [ ]:
track_features.shape

In [ ]:
track_features.describe()

In [ ]:
track_features.info()

#### Danceability
Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
#### Energy
Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
#### Key
The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1. >= -1 <= 11
#### Loudness
The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.
#### Mode
Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
#### Speechiness
Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
#### Acousticness
A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
#### Instrumentalness
Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
#### Liveness
Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
#### Valence
A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). >= 0 <= 1
#### Tempo
The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
#### Type
The type of body. Always "audio features".
#### ID
The Spotify ID for the track.
#### URI
The Spotify URI for the track.
#### Track_href
The URL for the song
#### Analysis_url
A URL to access the full audio analysis of this track. An access token is required to access this data.
#### Duration_ms
The duration of the track in milliseconds.
#### Time_signature
An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4". >= 3 <= 7

In [ ]:
dat_graph = track_features[graph_track_features]

In [ ]:
sns.pairplot(dat_graph)
plt.show()

In [ ]:
playlist_averages_query="""SELECT o.date, f.fake_name, 
                            AVG(s.acousticness) as avg_acousticness,
                            AVG(s.danceability) as avg_danceability,
                            AVG(s.duration_ms) as avg_duration_ms,
                            AVG(s.energy) as avg_energy,
                            AVG(s.instrumentalness) as avg_instrumentalness,
                            AVG(s.liveness) as avg_liveness,
                            AVG(s.loudness) as avg_loudness,
                            AVG(s.speechiness) as avg_speechiness,
                            AVG(s.tempo) as avg_tempo,
                            AVG(s.valence) as avg_valence
                        FROM owners o 
                        JOIN fake_names f ON o.owner_name=f.owner_name
                        JOIN playlist_songs ps ON o.playlist_id=ps.playlist_id
                        JOIN songs s ON ps.track_id=s.track_id
                        GROUP BY o.owner_name, f.fake_name, o.date
                        ORDER BY date ASC"""

In [ ]:
con = sqlite3.connect('spotify.db')
playlist_averages = pd.read_sql(playlist_averages_query,con,parse_dates=['date'])
con.close()

In [ ]:
playlist_averages.head()

In [ ]:
playlist_averages.date.dtype

In [ ]:
features_list = ['avg_acousticness',
                'avg_danceability',
                'avg_duration_ms',
                'avg_energy',
                'avg_instrumentalness',
                'avg_liveness',
                'avg_loudness',
                'avg_speechiness',
                'avg_valence']

In [ ]:
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(10,8))

counter = 0

for i in range(3):
    for j in range(3):
        sns.lineplot(ax=axs[i,j],data=playlist_averages,x='date',y=features_list[counter],hue='fake_name')
        if i == 0 and j == 0: 
            legend = axs[i,j].get_legend()
        axs[i,j].legend().remove()
        axs[i,j].xaxis_date()
        counter += 1

fig.legend(legend.get_lines(),playlist_averages.fake_name.unique(), bbox_to_anchor=(1.11, 0.95))
fig.autofmt_xdate()
fig.suptitle("Average Playlist Feature Scores per Month")
fig.tight_layout()
plt.show()